## DESE61003 - Audio Experience Design
# Week 4 - Visualising Sound (using Python)

Recognising sound often involves finding a suitable time-frequency representation. This is both important for yourself, to visually inspect the sound and get an idea of its frequency contents, and for the recognition algorithm you use to automate recognition, to give it the most useful starting point. However, time-frequency representations come with a set of parameters that can drastically alter the visualisation, and careful trade-offs need to be made. In this session, you'll get some practical experience to gain insight into this process.

In [ ]:
import numpy as np
from scipy import signal
import ipywidgets as widgets
from disiple.signals import AudioSignal, Spectrum, PowerSpectrum, Spectrogram
from disiple.operations import apply_adsr, apply_gain

We start by loading some useful packages and creating a couple of recurring variables.

In [ ]:
duration = 4
samplerate = 44100
nyquist_freq = samplerate / 2
time = np.arange(0, duration, 1/samplerate)

### 1. Frequency sweeps

Before we start properly, there is another type of basic signal that will come in useful later on when testing different visualisations. A _sine sweep_ or _chirp_ is a sinusoidal wave whose frequency changes steadily over time. You can create its samples using SciPy's [`signal.chirp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.chirp.html) function.

#### Activity 1

Create a couple of sine sweeps and listen to them. Try to create at least:

- a sweep of increasing frequency
- a sweep of decreasing frequency
- a logarithmically increasing sweep

In [ ]:
# your code here

#### Activity 2

- What happens if either parameter `f0` or `f1` has a value that is higher than the Nyquist frequency?
- What happens if parameter `t1` is set to a value that is different from the signal's duration?

In [ ]:
# your code here

_your answer here_

#### Activity 3

Plot the spectrum of a linear sweep from 0 to the Nyquist frequency. What does it remind you of and why is that?

In [ ]:
# your code here

_your answer here_

### 2. Zooming into a frequency range of interest

When we're analysing a piece of real-world audio, we are often mostly interested in the lower frequency regions. That does not mean that the high frequency content is superfluous for playback, on the contrary, it gives sound its brightness and "sparkle" as you can hear by comparing the song below with its downsampled version.

In [ ]:
song = AudioSignal('../data/Townhouse_Woods_-_Cat_Person.wav')
resampled_song = song.resample(8000)
song.play()
resampled_song.play()

Nonetheless, most natural sounds decrease in power further up in the frequency spectrum, and many high frequencies are actually harmonics of lower frequencies. This means that there is not much information in the higher frequency regions that is not already more apparent in the lower frequency regions.

In [ ]:
Spectrum(song).display()

For that reason, we often focus on a narrower range of interest when analysing the content of a sound signal. For instance, an upper limit of 4000 Hz means that only the fundamental frequency of the single highest [key on the piano](https://en.wikipedia.org/wiki/Piano_key_frequencies#List) ($C_8 = 4186$ Hz) cannot be represented, while most other instruments (including [soprano](https://en.wikipedia.org/wiki/Soprano) voice) produce far lower fundamental frequencies. This focussing on a particular range can be done literally by zooming into the spectrum.

In [ ]:
Spectrum(song).display(x_range=(0, 4000), y_range=(-140, -35))

However, calculating a spectrum that includes the higher frequencies just for them to be ignored afterwards is waste of computing power.

#### Activity 4

Can you think of a way to compute a spectrum with range 0-4000 Hz directly, without needing to throw away the high frequency bins? Try to demonstrate it by plotting the result.

_your answer here_

In [ ]:
# your code here

The frequency resolution $\Delta f$ determines how precisely we can distinguish between neighbouring frequencies, which is important for applications like music transcription and tuning.

#### Activity 5

Work out what the effect of downsampling is on the frequency resolution. Remember that $\Delta f = \dfrac{f_s}{N}$ with $f_s$ the sample rate and $N$ the length of the signal. Illustrate your answer with plots.

_your answer here_

In [ ]:
# your code here

### 3. The power of a signal and its spectrum

Plotting the spectrum of a signal is highly informative. One of the reasons is that the Fourier transform preserves power, a property that is known as [Parseval's theorem](https://en.wikipedia.org/wiki/Parseval%27s_theorem). Given a signal $x(n)$ of length $N$, we can calculate its (average) power $P_x$ as $$P_x = \dfrac{1}{N}\sum_{n=0}^{N-1} |x(n)|^2$$

We can get the power of any `AudioSignal` by calling the method `.power()`, which is by default expressed in decibels, but can be returned as a linear value by passing the `dB=False` argument. For instance, the power of a sine wave is -3 dB or 0.5.

In [ ]:
freq = 220
sine_signal = AudioSignal(np.sin(2*np.pi*freq*time), samplerate)
sine_signal.power(), sine_signal.power(dB=False)

#### Activity 6

What type of signal has maximal power and what is its value? Verify with code.

In [ ]:
# your code here

_your answer here_

We can use a similar method `.power()` on variables of type `Spectrum`, and verify that the Fourier transform indeed preserves power.

In [ ]:
sine_spectrum = Spectrum(sine_signal)
sine_spectrum.power(), sine_spectrum.power(dB=False)

What makes a spectrum so informative, is that we don't just get the power of the whole signal, but the power of individual frequency components. For this, we simply need to square the magnitudes of the (complex) spectrum. When we are expressing spectral values in dB, this is already included in the calculation (because decibels are only defined for power ratios), but if we want to calculate linear values of the so-called _power spectrum_ with the `disiple` library, we can either pass the argument `exponent=2` to `Spectrum` or directly create a variable of type `PowerSpectrum` (both do exactly the same).

In [ ]:
sine_power_spectrum = Spectrum(sine_signal, dB=False, exponent=2)
# OR
sine_power_spectrum = PowerSpectrum(sine_signal, dB=False)
sine_power_spectrum.display(x_range=(200, 240))

#### Activity 7

What strange phenomenon can you notice in the plot? Could you think of an explanation?

_your answer here_

Although mathematically correct, showing only half the signal's power in a plot of the spectrum can be counter-intuitive. Therefore it's possible to add an additional argument `norm_single_side_band=True` to `Spectrum` or `PowerSpectrum` that performs a rescaling such that the total power per frequency component can be easily read in the plot.

In [ ]:
PowerSpectrum(sine_signal, dB=False, norm_single_side_band=True).display(
    x_range=(200, 240), y_range=(0, 0.55),
    title='Spectrum of a sine wave normalised to show all power in the positive frequencies',
)
PowerSpectrum(sine_signal, dB=True, norm_single_side_band=True).display(
    x_range=(200, 240), y_range=(-20, 0),
    title='Spectrum of a sine wave normalised to show all power in the positive frequencies',
)

With this option, it is immediately clear that the power of our sine wave is 0.5 or 3 dB, as expected.

Observing this distribution of power over different frequencies becomes more useful whent plotting more complex signals, such as a sequence of two sines of different frequencies (which is created by concatenating multiple `AudioSignal`s with the `&` operator).

In [ ]:
@widgets.interact(ratio=widgets.IntSlider(min=1, max=10, step=1, value=2, description='Ratio of sine durations',
                                           style={'description_width': 'initial'}, layout=widgets.Layout(width='50%')))
def sequential_signals(ratio):
    sine2_length = round(len(sine_signal)/ratio)
    sine2_signal = AudioSignal(np.sin(2*np.pi*2*freq*time[:sine2_length]), samplerate)
    seq_signal = sine_signal & sine2_signal
    seq_signal.play()
    seq_spectrum = PowerSpectrum(seq_signal, norm_single_side_band=True)
    seq_spectrum.display(
        title='Spectrum of two sines in sequence with a duration ratio of {}:1'.format(ratio),
        x_range=(freq-50, 2*freq+50), y_range=(-25, 0)
    )
    print(f'The average power in the whole signal is {seq_signal.power():.1f} dB')
    delta_freq = seq_signal.samplerate / len(seq_signal)
    freq_idx = round(freq / delta_freq)
    two_freq_idx = round(2 * freq / delta_freq)
    print(f'The power at {freq} Hz is {seq_spectrum.magnitude[freq_idx]:.1f} dB and the power at {2*freq} Hz is {seq_spectrum.magnitude[two_freq_idx]:.1f} dB')

#### Activity 8

Play around with the interactive example above which contains a first sine of fixed duration, followed by a second sine of double the frequency and a duration that's controlled by the slider. The slider set to 2 will make the second sine half the length of the first, and so on. Observe how the power of both sine components in the spectrum fluctuates with the duration ratio. What is the relationship between the duration ratio and the power ratio of the two sines? Why does the power of the first sine change although its duration is fixed?

_your answer here_

#### Activity 9

Because a spectrum is calculated for the whole length of a signal and therefore discards all temporal information, there are multiple different signals that have the same spectrum. Can you create another signal that has the same spectrum as the sequence of two sines above? Don't bother making your answer interactive, just pick a single value for the power ratio as long as it's not 1.

_your answer here_

In [ ]:
# your code here

### 4. The difference between best-case and real-world scenarios

In all examples given so far, the exact frequency and power of the sine signal has been easy to read from the spectrum. Unfortunately, that was because these sine signals were all special cases. Remember that a spectrum computed from a digital waveform is a discrete signal itself, which is defined for specific frequencies only, namely integer multiples of $\Delta f$. The sine waves we've created so far all happened to have a frequency that matched one of the points of this discrete frequency axis.

In [ ]:
freq in sine_spectrum.frequencies

How does the spectrum look like then for a sine with a frequency that falls between these discrete points? As you can imagine, its power is spread over surrounding frequency points or _bins_. Not just the directly adjacent ones, although those are most affected, but also some that are further away. This phenomenon is known as _spectral leakage_.

In [ ]:
freq = 220*2**(7/12)
sine_signal = AudioSignal(np.sin(2*np.pi*freq*time), samplerate)
sine_signal.power(), sine_signal.power(dB=False)
sine_power_spectrum = PowerSpectrum(sine_signal, dB=False, norm_single_side_band=True)
sine_power_spectrum.display(
    x_range=(freq-20, freq+20), y_range=(0, 0.25),
    title='Spectrum of a sine wave with a frequency that is not an exact multiple of Δf',
)
print(f'The power of the sine is {sine_power_spectrum.power(dB=False):.1f}')

It becomes therefore harder to verify both what the exact frequency and exact power of the sine wave are. Note that the total power in the spectrum is the same whether the sine frequency falls on a bin frequency or not.

While the leaking of the sinusoidal peak into the surrounding bins is still relatively predictable for isolated sines (and interpolation strategies exist to reconstruct the exact peak), the view becomes more muddled when multiple frequencies are involved. Especially since we also have the spectral resolution to deal with, as will become apparent in the next example.

In [ ]:
freq = 220
delta_freq = samplerate / len(time)
base_sine = AudioSignal(0.5 * np.sin(2*np.pi*freq*time), samplerate)
close_sine = AudioSignal(0.5 * np.sin(2*np.pi*(freq+1.5*delta_freq)*time), samplerate)
too_close_sine = AudioSignal(0.5 * np.sin(2*np.pi*(freq+0.5*delta_freq)*time), samplerate)
indistinguishable_mix = base_sine + too_close_sine
distinguishable_mix = base_sine + close_sine

In the snippet above, we've created three sine waves with close frequencies and combine them into two mixtures. The first mixture contains two sines whose frequency difference is larger than $\Delta f$, whereas the second mixture contains two sines with a frequency difference below $\Delta f$. Although the difference between the individual sines is barely noticeable, their combinations are notably different (a temporal evolution of their amplitudes is visible due to the two sines periodically cancelling each other out)

In [ ]:
base_sine.play()
base_sine.display(title='Single sine wave')
indistinguishable_mix.play()
indistinguishable_mix.display(title='Combination of two sines with frequency separation below Δf')
distinguishable_mix.play()
distinguishable_mix.display(title='Combination of two sines with frequency separation above Δf')

When we look at the spectra of the mixed signals, it is hard to decide how many sines are present in the signal. As you'd expect for the two sines that are closer to each other than $\Delta f$, they are fused together into one wide peak, but similar wide peaks appear for single sines whose frequency is not a multiple of $\Delta f$. What's worse is that even the mixture of two sines with frequencies separated by more than $\Delta f$ does not give two clear peaks. Because the second peak falls between discrete frequency points, part of it gets distributed to the first peak, again creating a single fused peak. In practice it can therefore be very hard to distinguish between two audibly distinct signals based solely on their spectrum.

In [ ]:
Spectrum(base_sine, dB=False).display(x_range=(freq-2, freq+2), y_range=(0, 0.3), title='Spectrum of a single sine wave')
Spectrum(indistinguishable_mix, dB=False).display(
    x_range=(freq-2, freq+2), y_range=(0, 0.3), 
    title='Spectrum of a combination of two sines with frequency separation below Δf',
)
Spectrum(distinguishable_mix, dB=False).display(
    x_range=(freq-2, freq+2), y_range=(0, 0.3), 
    title='Spectrum of a combination of two sines with frequency separation above Δf',
)

#### Activity 10

Rerun the example above for a number of sines with frequencies that both coincide or not with the discrete frequency bins. Can you find frequencies for which two separate peaks are clearly visible?

_your answer here_

### 5. Visualising time-varying signals

Because we can only derive the power of the frequencies in the _entire_ signal from a spectrum, without possibility to distinguish between power inequalities caused by amplitude or duration differences, it is most useful for signals that don't vary over time (or if we don't care about temporal variations). In case we do want to observe the variation over time, we can use another signal representation, the _spectrogram_.

Conceptually, the idea of a spectrogram is relatively simple. You divide the signal in multiple shorter segments and calculate a spectrum for each of them. It does however introduce additional parameters that can significanty alter the visualisation and therefore need careful consideration.

The first relevant parameter is the width of each segment. Since we call these segments _frames_, the width is also know as the _frame width_, which we express in samples. We create a spectrogram with the `disiple` library by passing a signal to a `Spectrogram` constructor and set the option `frame_size` to an integer, 2048 below.

In [ ]:
up_down_sweep = AudioSignal(signal.chirp(t=time, f0=20, t1=duration, f1=samplerate), samplerate)
up_down_sweep.play()
specgram = Spectrogram(up_down_sweep, frame_size=2048)
specgram.display(title='Spectrogram with a frame size of 2048')

In the resulting picture, we can now clearly see the temporal evolution of the sine sweep, first going up and then coming down again (created by intentionally using aliasing). Each of the columns in the picture is a spectrum of a frame, which we can show individually.

In [ ]:
spectrum_at_1s = specgram.spectrum_at(time=1)
spectrum_at_1s.display(title='Spectrum of the frame at 1 s')

#### Activity 11

Since each column of the spectrogram is simply a spectrum, all the real-world challenges we've seen so far apply to spectrograms as well. An extra complication is that in practice, the frame size will be short compared to the length of a typical signal. This makes the frequency resolution $\Delta f$ become significant for yet another reason. Can you think of why that is? Hint: it imposes another condition on the signals we can represent.

_your answer here_

On the one hand, we'd like to make our frames wide, since it improves the frequency resolution. On the other hand, a wide frame has the same drawback as a spectrum: we don't know when exactly during the frame the computed frequencies are present. Therefore a fine frequency resolution leads to a coarse _time resolution_. Even though we'd like them to be as fine as possible, we cannot have both since they are inversely related, which is a manifestation of [Heisenberg's uncertainty principle](https://en.wikipedia.org/wiki/Uncertainty_principle#Signal_processing).

In [ ]:
narrowband_specgram = Spectrogram(up_down_sweep, frame_size=8192)
narrowband_specgram.display(title='Spectrogram with a frame size of 8192')

Picking the right frame size for a particular signal and application is therefore always a trade-off. For instance, the spectrogram above is calculated with wider frames of 8192 samples (leading to a smaller $\Delta f$ and narrower frequency bands). However, because the signal is a frequency sweep, it varies strongly over time (known as being _non-stationary_). Therefore the wider frame contains more different frequencies, which all get blended together. Together with the frequencies leaking into surrounding bins, this actually makes the precise frequency determination harder, despite the increased frequency resolution.

For convenience, to make control over the frequency and time resolution more explicit regardless of sample rate, it is also possible to specify the frame width as a duration in seconds with the `frame_duration` option, as shown below.

In [ ]:
tenHz_specgram = Spectrogram(up_down_sweep, frame_duration=0.100)
tenHz_specgram.display(title='Spectrogram with a frame duration of 100 ms (therefore frequency resolution of 10 Hz)')

Relatively independent of the frame width, we also need to specify the distance between two consecutive frames. Similarly to the frame width, we have the choice of passing this distance as an option to the `Spectrogram` constructor either as a size in samples with `step_size` or as a duration in seconds with `step_duration`.

Setting `step_size` equal to the `frame_size` will create contiguous, non-overlapping segments, which can be a good idea if your signal consists of regular events and you don't want to analyse across their boundaries, e.g. you want to slice a piece of music along its beats (assuming that you know the tempo and know it to be very regular). Having a `step_size` larger than the `frame_size` will leave gaps in the signal that are not included in any frame, so that's rarely a good idea. In most cases though, a signal will consist of temporal events that are irregular or have an unknown tempo.

Suppose that you have a signal that contains some event that can occur at any time and that you want to recognise. Examples of such events include music notes or words. Even if you've set the frame width large enough to contain the whole event, using non-overlapping segments does not ensure that the whole event is captured in a single frame. It could be that the event only starts halfway a frame, with its latter half ending up in the next frame, but any possible split of the event between frames is possible. If you rely on having the whole event in a single frame for its recognition, for instance for recognising a keyword, this will lead to errors.

The solution is to let frames overlap. The most extreme case is to advance only a single sample between frames, which is known as a _sliding window_. Doing so means that there always will be a frame that aligns with the start of your event. Note that in this case there will still be many other frames that contain (parts of) the event starting at a later point in the frame, which might confuse your recognition or lead to recognising multiple instances. In other situations, you don't need the entire event in a single frame to be recognised. For instance, you can detect the presence of a musical note based on only a fragment. In that case, you can use larger step size, but still might want to have some overlap to ensure that there's at least a frame containing the majority of the event.

A good default is to use a shift size that is half the frame size, which is what's used when no step is specified in `Spectrogram`. When applying a windowing function to each frame, which is commonly done to reduce spectral leakage, a step of maximum half the frame size is necessary such that each sample appears near the middle of at least one frame, where the sample value remains mostly unchanged.

Another way to look at the step size is that it determines the sample rate of the spectrogram signal. You can consider a spectrogram as a new time signal, which is no longer one-dimensional with a single amplitude per point in time, but multi-dimensional giving values for a fixed number of bins at each time point. Having a step size of 1 will cause the rate of the spectrogram to be equal to the audio signal's sample rate, any higher step size will make the spectrogram have a lower rate. This reduction of sample rate can be welcome, as it reduces the amount of data that needs to be processed or stored. For instance, suppose we want to use a spectrogram to determine what music notes are being played. It is probably excessive to localise the exact start of a note to the microsecond (which is the temporal resolution you get for common audio sample rates), as we don't perceive such small differences anyway. Instead we can work with spectrograms calculated every millisecond and save some computational effort by choosing a step size in the order of 100.


In conclusion, although the step between frames can theoretically be changed independently of the frame width, in practice it needs to be tuned together with the frame width to a particular type of signal and application.

#### Activity 12

Play around with the `step_size` or `step_duration` options of `Spectrogram` and see how it changes the spectrogram. Note that drawing spectrograms with a large number of points can take some time. A good part of the time is spent creating the tooltips, which you can disable by passing `tooltips=None` to the `.display()` method in order to speed up drawing. Nonetheless, very detailled spectrograms with wide frames and small steps will likely cause your browser to run out of memory. So decrease `step_size` in small amounts instead of immediately setting it to 1.

In [ ]:
# your code here

### 6. The spacing of frequency bins

A final option in adapting the visualisation of a signal to a specific purpose is the distribution of frequency bins. The Fourier transform provides frequencies that are linearly spaced on the discrete frequency axis. However, that does not match the perception of our hearing, nor the [harmonic series](https://en.wikipedia.org/wiki/Harmonic_series_(music)). Therefore it  makes sense for some signals to display frequencies on a logarithmic axis.

We achieve this by passing the option `spacing='log'` to the `Spectrogram` constructor. Because a logarithmic axis cannot start with a frequency of 0 Hz (which would map to $-\infty$), we need to explicitly specify a minimum frequency as `min_freq`. This minimum frequency needs to be high enough to fit into the given frame width, and will throw an error if this is not the case. Alternatively, you can let the minimum frequency determine the frame width, by making the latter just wide enough to fit a single period of this frequency. Then you don't need to pass the frame width in any other way. Beware though, when used in this way, the minimum frequency also determines the frequency resolution, so make sure to keep it low.

You can optionally set a maximum frequency with `freq_max` to zoom in on a specific frequency range. By default it is set to the Nyquist frequency. Finally, you also need to specify the number of bins per semitone with `num_bins`. A value of 1 calculates frequency bins that coincide with music notes, a value of 2 places an extra bin between each music note and so on.

In [ ]:
log_specgram = Spectrogram(up_down_sweep, spacing='log', min_freq=27.5, num_bins=1)
log_specgram.display(title='Logarithmically spaced spectrogram of a linearly increasing sine sweep')

In [ ]:
zoomed_log_specgram = Spectrogram(up_down_sweep, frame_duration=1/20, spacing='log', min_freq=1760, max_freq=3520, num_bins=3)
zoomed_log_specgram.display(title='Logarithmically spaced spectrogram with frequencies between 1760 and 3520 Hz')

If we'd be only interested in _displaying_ frequencies on a logarithmic axis, it would suffice to use standard plotting functionality to interpolate linear spectrogram pixels onto a logarithmic axis. Instead we want to calculate the actual logarithmic spectrum values such that they can be used for further processing (and display the corresponding pixel values on an equidistant grid). For this implementation, a filterbank that redistributes the power of the linear bins over the appropriate logarithmic bins is used. This two-stage process is simple, but also means that reconstruction of the signal from a logarithmic spectrogram is impossible, whereas it is possible for a linear spectrogram (under certain constraints for its parameters). Direct computations of a logarithmic spectrogram exist, which are invertible, but they are computationally more intensive.

#### Activity 13

Create a signal of multiple harmonic signals with different fundamental frequencies in sequence. Plot spectrograms with both linear and logarithmic frequency axes and appropriate parameters. Zoom in on the harmonic frequency patterns. Wat do you notice about way these patterns are visualised in the different spectrograms and why might one be preferred over the other? You might want to create a function that creates a harmonic signal for a given frequency to avoid duplicating code. Or if you're in a hurry, you can use a triangular wave.

In [ ]:
# your code here

_your answer here_